# Simulações de Eletrolisadores — Análises com descrições e fontes (v7)

Este caderno traz as simulações **I–V**, **Eficiência×Temperatura**, **Sobretensões×T** e **Dinâmica**,
com **descrições didáticas** e, logo após cada análise, uma seção **“Leitura recomendada e fontes (URL + DOI)”**
que aponta onde o leitor pode ver a fundamentação teórica, as fórmulas e aplicações correlatas.

## Referências-base (links estáveis, preferir DOI)
- **ZENG, K.; ZHANG, D.** Recent progress in alkaline water electrolysis for hydrogen production and applications.
  *Progress in Energy and Combustion Science*, 36(3), 307–326, 2010. **DOI:** https://doi.org/10.1016/j.pecs.2009.11.002  
- **CARMO, M.; FRITZ, D. L.; MERGEL, J.; STOLTEN, D.** A comprehensive review on PEM water electrolysis.
  *International Journal of Hydrogen Energy*, 38(12), 4901–4934, 2013. **DOI:** https://doi.org/10.1016/j.ijhydene.2013.01.151  
- **LAGUNA‑BERCERO, M. A.** Recent advances in high temperature electrolysis using SOECs.
  *Journal of Power Sources*, 203, 4–16, 2012. **DOI:** https://doi.org/10.1016/j.jpowsour.2011.12.019  
- **de GROOT, M. T.; KRAAKMAN, J.; BARROS, R. L. G.** Optimal operating parameters for advanced alkaline water electrolysis.
  *International Journal of Hydrogen Energy*, 47(82), 15487–15510, 2022. **DOI:** https://doi.org/10.1016/j.ijhydene.2022.08.075  
- **GAMBOU, F. et al.** A Comprehensive Survey of Alkaline Electrolyzer Modeling.
  *Energies*, 15(9):3452, 2022. **DOI:** https://doi.org/10.3390/en15093452  
- **SONE, Y.; EKDUNGE, P.; SIMONSSON, D.** Proton conductivity of Nafion 117 (four‑electrode AC impedance).
  *Journal of the Electrochemical Society*, 143:1254–1259, 1996. **DOI:** https://doi.org/10.1149/1.1836625  
- **AZUAN, M. et al.** Effect of temperature on performance of advanced alkaline electrolyzer.
  *Science International (Lahore)*, 31(5):757–762, 2019. **PDF:** https://www.sci-int.com/pdf/637054513794717651.pdf  
- **BI, X. et al.** Simulation study on the effect of temperature… *Fuel*, 380:133209, 2025.
  **DOI:** https://doi.org/10.1016/j.fuel.2024.133209  
- **ERA5‑Land** monthly means (1991–2020). **DOI:** https://doi.org/10.24381/cds.68d2bb30  
- **INMET** Normais 1991–2020 (Brasil). **URL:** https://portal.inmet.gov.br/  
- **BARD, A. J.; FAULKNER, L. R.** Electrochemical Methods: Fundamentals and Applications, 2nd ed. Wiley, 2000.
  **Página do livro:** https://www.wiley.com/en-us/Electrochemical+Methods%3A+Fundamentals+and+Applications%2C+2nd+Edition-p-9780471043720  
- **NEWMAN, J.; THOMAS‑ALYEA, K.** Electrochemical Systems, 3rd ed. Wiley, 2004.
  **Página do livro:** https://www.wiley.com/en-us/Electrochemical+Systems%2C+3rd+Edition-p-9780471470755

### Funções do modelo (Nernst, Tafel/Butler–Volmer simplificado, perdas ôhmicas e de concentração)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
R = 8.314462618
F = 96485.33212
plt.rcParams.update({"figure.dpi": 140, "font.size": 12, "axes.grid": True})

def nernst_voltage(T_K, p_H2=1.0, p_O2=1.0, p_H2O=1.0):
    V0 = 1.229
    V_T = V0 - 0.0009*(T_K - 298.15)
    return V_T + (R*T_K)/(2*F)*np.log((p_H2*(p_O2**0.5))/max(p_H2O,1e-12))

def tafel_eta(i, i0, T_K, alpha=0.5):
    i = np.maximum(i, 1e-12)
    return (R*T_K)/(alpha*F)*np.log(i/np.maximum(i0, 1e-20))

def ohmic_eta(i, ASR):
    return i*ASR

def concentration_eta(i, i_lim, T_K):
    eps = 1e-12
    ratio = np.clip(1 - i/np.maximum(i_lim, eps), eps, 1-1e-9)
    return (R*T_K)/(2*F)*np.log(1/ratio)

def v_cell(i, params, T_K):
    Vrev = nernst_voltage(T_K, params.get("p_H2",1), params.get("p_O2",1), params.get("p_H2O",1))
    eta_act = tafel_eta(i, params["i0"](T_K), T_K, params.get("alpha",0.5))
    eta_ohm = ohmic_eta(i, params["ASR"](T_K))
    eta_conc = concentration_eta(i, params["i_lim"](T_K), T_K)
    return Vrev + eta_act + eta_ohm + eta_conc, (Vrev, eta_act, eta_ohm, eta_conc)

def faradaic_efficiency(i):
    return 0.985 - 0.02*np.exp(-i/0.2)

def energy_efficiency(Vrev, Vcell, eta_F=0.985):
    return (Vrev/np.maximum(Vcell, 1e-12))*eta_F

def arrhenius(val_25C, Ea_kJmol, T_K):
    Ea = Ea_kJmol*1000.0
    return val_25C*np.exp(-Ea/R*(1/T_K - 1/298.15))

# Parametrizações base e dependências com T (tendências)
def ASR_AWE(T_K):
    base = 0.24 - 0.0012*(T_K-298.15)
    penalty = 0.0008*max(T_K-348.15, 0.0)
    return float(np.clip(base + penalty, 0.06, 0.30))
def i0_AWE(T_K):  return float(arrhenius(2e-6, 35.0, T_K))
def i_lim_AWE(T_K): return float(2.6 + 0.002*(T_K-298.15))

def ASR_PEM(T_K):
    base = 0.20 - 0.0010*(T_K-298.15)
    penalty = 0.0010*max(T_K-343.15, 0.0)
    return float(np.clip(base + penalty, 0.05, 0.25))
def i0_PEM(T_K):   return float(arrhenius(1e-5, 28.0, T_K))
def i_lim_PEM(T_K): return float(2.4 + 0.0015*(T_K-298.15))

def ASR_SOEC(T_K): return float(0.02 + 0.06*np.exp(-(T_K-873.15)/220.0))
def i0_SOEC(T_K):  return float(arrhenius(5e-4, 12.0, T_K))
def i_lim_SOEC(T_K): return float(3.0 + 0.002*(T_K-298.15))

PARAMS = {
    "AWE": {"ASR": ASR_AWE, "i0": i0_AWE, "i_lim": i_lim_AWE, "alpha": 0.5, "p_H2":1, "p_O2":1, "p_H2O":1},
    "PEM": {"ASR": ASR_PEM, "i0": i0_PEM, "i_lim": i_lim_PEM, "alpha": 0.5, "p_H2":1, "p_O2":1, "p_H2O":1},
    "SOEC":{"ASR": ASR_SOEC,"i0": i0_SOEC,"i_lim": i_lim_SOEC,"alpha": 0.5, "p_H2":1, "p_O2":1, "p_H2O":1},
}

def make_scaled(param, a_scale, i0_scale):
    def ASR_scaled(T): return a_scale*param["ASR"](T)
    def i0_scaled(T):  return i0_scale*param["i0"](T)
    return {"ASR": ASR_scaled, "i0": i0_scaled, "i_lim": param["i_lim"],
            "alpha": param["alpha"], "p_H2":1, "p_O2":1, "p_H2O":1}

def calibrate_params(target_eta=0.80, T_star=333.15, i_star=1.2, param=None, max_iter=24):
    a_s, i0_s = 1.0, 1.0
    for _ in range(max_iter):
        trial = make_scaled(param, a_s, i0_s)
        Vcell,(Vrev, *_)= v_cell(i_star, trial, T_star)
        eta = energy_efficiency(Vrev, Vcell, eta_F=0.985)
        if eta < target_eta:
            a_s *= 0.92
            i0_s *= 1.6
        else:
            a_s *= 1.02
            i0_s *= 0.90
    return make_scaled(param, a_s, i0_s)

PARAMS_USE = dict(PARAMS)
PARAMS_USE['AWE'] = calibrate_params(target_eta=0.824, T_star=348.15, i_star=1.2, param=PARAMS['AWE'])
PARAMS_USE['PEM'] = calibrate_params(target_eta=0.796, T_star=343.15, i_star=1.2, param=PARAMS['PEM'])

**Onde está no código (equações):**  
  - **Nernst**: `nernst_voltage(T_K, ...)`  
  - **Tafel/Butler–Volmer simplificado**: `tafel_eta(i, i0, T_K, alpha)`  
  - **Perda ôhmica**: `ohmic_eta(i, ASR)`  
  - **Sobretensão de concentração** (limite de transporte): `concentration_eta(i, i_lim, T_K)`  
  As tendências de $i_0(T)$ e $\mathrm{ASR}(T)$ seguem **Arrhenius** e literatura citada.

## 1) Curvas de polarização (I–V) — AWE/PEM/SOEC

**Descrição rápida.** A curva I–V mostra três regimes: (i) **ativação** (cinética), (ii) **ôhmico** (resistências do conjunto) e (iii) **concentração** (limites de transporte). Em geral, **SOEC** (alta T) apresenta menores perdas para mesma densidade de corrente; **PEM** tende a operar com menores tensões que **AWE** em faixas equivalentes de $i$ e $T$.

In [ ]:
i_grid = np.linspace(0.2, 2.5, 180)
T_K = 333.15  # 60 °C
fig, ax = plt.subplots(figsize=(6.6,4.2))
for tech,label in [("AWE","Alkaline"),("PEM","PEM"),("SOEC","SOEC")]:
    Vcell,_ = v_cell(i_grid, PARAMS_USE[tech], T_K)
    ax.plot(i_grid, Vcell, label=f"{label} eletrolizer")
ax.set_xlabel("Current density (A/cm$^2$)"); ax.set_ylabel("Cell Voltage (V)"); ax.legend()
plt.tight_layout(); plt.show()

**Interpretação.** As curvas geradas reproduzem o padrão esperado de literatura: **AWE** apresenta maior parcela ôhmica; **PEM** tende a operar com tensão um pouco menor; **SOEC** desloca a curva para baixo pela operação a **alta temperatura**.

**Leitura recomendada e fontes (URL + DOI):**
- **ZENG, K.; ZHANG, D.** Progress in Energy and Combustion Science, 2010. **DOI:** https://doi.org/10.1016/j.pecs.2009.11.002  — Visão geral de AWE, curvas I–V, perdas e operação.
- **CARMO, M. et al.** International Journal of Hydrogen Energy, 2013. **DOI:** https://doi.org/10.1016/j.ijhydene.2013.01.151  — Revisão abrangente de **PEM** (material de membrana, cinética, perdas e desempenho típico).
- **LAGUNA‑BERCERO, M.A.** Journal of Power Sources, 2012. **DOI:** https://doi.org/10.1016/j.jpowsour.2011.12.019  — Revisão de **SOEC** e vantagens em alta temperatura.
- **BARD & FAULKNER (livro).** Fundamentos de **Nernst** e **Butler–Volmer/Tafel**: https://www.wiley.com/en-us/Electrochemical+Methods%2C+2nd+Edition-p-9780471043720
- **NEWMAN & THOMAS‑ALYEA (livro).** Modelagem de polarização e transporte: https://www.wiley.com/en-us/Electrochemical+Systems%2C+3rd+Edition-p-9780471470755

## 2) Eficiência energética × Temperatura — AWE/PEM (SOEC como referência)

**Descrição rápida.** A eficiência tende a **aumentar com a temperatura** até um **ótimo** (tipicamente **70–75 °C** em AWE/PEM), devido ao aumento de $i_0(T)$ e à redução de $\mathrm{ASR}(T)$; acima disso, limitações de **materiais** e **gestão de água** podem reduzir ganhos. Em **SOEC**, a alta T (700–850 °C) favorece fortemente a cinética e o transporte.

In [ ]:
T_list = np.linspace(308.15, 353.15, 11)  # 35–80 °C
i_op = 1.2
eff = {"AWE":[], "PEM":[], "SOEC":[]}
for T in T_list:
    for tech in ["AWE","PEM"]:
        Vcell,(Vrev, *_)= v_cell(i_op, PARAMS_USE[tech], T)
        eff[tech].append(100*energy_efficiency(Vrev, Vcell, eta_F=faradaic_efficiency(i_op)))
T_SOEC = 1123.15  # 850 °C
Vcell,(Vrev, *_)= v_cell(i_op, PARAMS_USE["SOEC"], T_SOEC)
eta_SOEC = 100*energy_efficiency(Vrev, Vcell, eta_F=faradaic_efficiency(i_op))
eff["SOEC"] = [eta_SOEC for _ in T_list]

fig, ax = plt.subplots(figsize=(6.6,4.2))
ax.plot(T_list-273.15, eff["AWE"], marker="o", label="Alkaline"); 
ax.plot(T_list-273.15, eff["PEM"], marker="o", label="PEM");
ax.plot(T_list-273.15, eff["SOEC"], ls="--", label="SOEC @ 850°C");
ax.set_xlabel("Temperatura (°C)"); ax.set_ylabel("Eficiência (%)"); ax.legend(); plt.tight_layout(); plt.show()

**Interpretação.** O ganho de eficiência com T (até o ótimo) aparece para **AWE** e **PEM**, enquanto a referência **SOEC @ 850 °C** destaca a vantagem térmica em alta temperatura.

**Leitura recomendada e fontes (URL + DOI):**
- **CARMO, M. et al.** (PEM) **DOI:** https://doi.org/10.1016/j.ijhydene.2013.01.151 — Efeitos de T na membrana, gestão de água e desempenho.
- **ZENG, K.; ZHANG, D.** (AWE) **DOI:** https://doi.org/10.1016/j.pecs.2009.11.002 — Condutividade de eletrólitos alcalinos e influência de T.
- **de GROOT, M. T. et al.** (AWE, ótimos operacionais) **DOI:** https://doi.org/10.1016/j.ijhydene.2022.08.075 — Parâmetros ótimos e trade‑offs.
- **LAGUNA‑BERCERO, M.A.** (SOEC) **DOI:** https://doi.org/10.1016/j.jpowsour.2011.12.019 — Vantagens de alta T.
- **AZUAN, M. et al.** (AWE, T vs desempenho) **PDF:** https://www.sci-int.com/pdf/637054513794717651.pdf
- **BI, X. et al.** (AWE, estudo de simulação) **DOI:** https://doi.org/10.1016/j.fuel.2024.133209
- **SONE, Y. et al.** (condutividade da Nafion vs T) **DOI:** https://doi.org/10.1149/1.1836625

## 3) Distribuição de sobretensões × Temperatura — AWE

**Descrição rápida.** A **ativação** cai com T (cinética mais rápida, $i_0\uparrow$); a **ôhmica** tende a cair com maior condutividade do eletrólito/partes; a **concentração** torna‑se relevante ao aproximar‑se do **limite de transporte**.

In [ ]:
T_list = np.linspace(298.15, 353.15, 12)
i_op = 1.5
fracs = []
for T in T_list:
    Vcell,(Vrev, act, ohm, conc)= v_cell(i_op, PARAMS_USE["AWE"], T)
    total = max(act+ohm+conc, 1e-9)
    fracs.append([100*act/total, 100*ohm/total, 100*conc/total])
fracs = np.array(fracs)

fig, ax = plt.subplots(figsize=(6.6,4.2))
ax.stackplot(T_list-273.15, fracs[:,0], fracs[:,1], fracs[:,2], labels=["Ativação","Ôhmica","Concentração"])
ax.set_xlabel("Temperatura (°C)"); ax.set_ylabel("Distribuição de Sobretensão (%)"); ax.legend(loc="upper right")
plt.tight_layout(); plt.show()

**Interpretação.** Na faixa de correntes e temperaturas empregada, as frações **ativação** e **ôhmica** diminuem com T; **concentração** cresce à medida que $i$ se aproxima de $i_{\text{lim}}$.

**Leitura recomendada e fontes (URL + DOI):**
- **GAMBOU, F. et al.** Modelagem detalhada de AWE (inclui condutividade específica de eletrólito). **DOI:** https://doi.org/10.3390/en15093452
- **CARMO, M. et al.** Revisão de perdas e operação de PEM. **DOI:** https://doi.org/10.1016/j.ijhydene.2013.01.151
- **BARD & FAULKNER.** Equações de **Butler–Volmer/Tafel** e decomposição de perdas: https://www.wiley.com/en-us/Electrochemical+Methods%2C+2nd+Edition-p-9780471043720
- **NEWMAN & THOMAS‑ALYEA.** Formulações de transporte e limites de concentração: https://www.wiley.com/en-us/Electrochemical+Systems%2C+3rd+Edition-p-9780471470755

## 4) Dinâmica (degraus de corrente) — AWE — 1ª ordem e balanço de energia

**Descrição rápida.** O modelo de **1ª ordem** representa a inércia térmica agregada via constante de tempo; o **balanço de energia** ($C_{th}\,\tfrac{dT}{dt}=P_{loss}-h\,(T-T_{env})$) acopla **perdas elétricas** ($i\,(V_{cell}-V_{rev})$) com a **troca térmica** (coeficiente $h$) e a **capacidade térmica** $C_{th}$ do conjunto.

In [ ]:
t = np.arange(0, 60*60+1, 10)
steps_Acm2 = [0.7, 0.6, 1.0, 0.9, 1.2, 1.1, 1.4]
t_step = np.linspace(5, 55, len(steps_Acm2))*60
i_series = np.ones_like(t, dtype=float)*steps_Acm2[0]
for k, tk in enumerate(t_step[1:], start=1):
    i_series[t>=tk] = steps_Acm2[k]

T_base = 333.15; tau = 15*60
T1 = np.zeros_like(t, dtype=float); T1[0] = T_base
for k in range(1, len(t)):
    T_target = T_base + 4.0*(i_series[k]-np.mean(steps_Acm2))
    T1[k] = T1[k-1] + ((T_target - T1[k-1]) * (t[k]-t[k-1]) / tau)

T_env = 298.15; h = 0.6; C_th = 12.0
T2 = np.zeros_like(t, dtype=float); T2[0] = T_base
for k in range(1, len(t)):
    Vcell,(Vrev, *_)= v_cell(max(i_series[k-1],1e-6), PARAMS_USE["AWE"], T2[k-1])
    P_loss = max(i_series[k-1]*(Vcell - Vrev), 0.0)
    dTdt = (P_loss - h*(T2[k-1]-T_env)) / C_th
    T2[k] = T2[k-1] + dTdt*(t[k]-t[k-1])

def efficiency_series(t, i_series, T_series, param):
    eff = np.zeros_like(t, dtype=float)
    for k, (Ti, ik) in enumerate(zip(T_series, i_series)):
        Vcell,(Vrev, *_)= v_cell(max(ik,1e-6), param, Ti)
        eff[k] = 100*energy_efficiency(Vrev, Vcell, eta_F=0.985)
    return eff

eff1 = efficiency_series(t, i_series, T1, PARAMS_USE["AWE"])
eff2 = efficiency_series(t, i_series, T2, PARAMS_USE["AWE"])

fig, ax1 = plt.subplots(figsize=(7.2,4.2))
ax1.plot(t/60.0, eff1, label="Eficiência (%) — 1ª ordem")
ax1.plot(t/60.0, eff2, label="Eficiência (%) — balanço de energia")
ax1.set_xlabel("Tempo (min)"); ax1.set_ylabel("Eficiência Energética (%)")
ax2 = ax1.twinx(); ax2.plot(t/60.0, i_series, ls="--", drawstyle="steps-post", label="Densidade de Corrente (A/cm²)")
ax2.set_ylabel("Densidade de Corrente (A/cm²)")
lines, labels = ax1.get_legend_handles_labels(); lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines+lines2, labels+labels2, loc="lower left")
plt.tight_layout(); plt.show()

**Interpretação.** As duas abordagens convergem para médias semelhantes, mas o **balanço de energia** reage de acordo com as perdas instantâneas e a convecção, capturando melhor efeitos de **i** e **T**. Em setups AWE, constantes de tempo **dezenas de minutos** são comuns; em PEM, **poucos minutos**.

**Leitura recomendada e fontes (URL + DOI):**
- **CARMO, M. et al.** (dinâmica e operação PEM) **DOI:** https://doi.org/10.1016/j.ijhydene.2013.01.151
- **ZENG, K.; ZHANG, D.** (AWE; operação e perdas) **DOI:** https://doi.org/10.1016/j.pecs.2009.11.002
- **O’Hayre, R. et al.** Fuel Cell Fundamentals, 4th ed., Wiley, 2016 — dinâmica e transientes de pilhas/eletrolisadores. Página do livro: https://www.wiley.com/en-us/Fuel+Cell+Fundamentals%2C+4th+Edition-p-9781119113803

## 5) Análise climática e tropicalização — Fortaleza vs regiões usuárias

**Descrição rápida.** Usar **temperaturas ambientes elevadas e pouco sazonais** (como em Fortaleza) reduz a energia de partida/aquecimento e as perdas térmicas do balanço de energia, melhorando a eficiência global (até os limites de materiais). Em contrapartida, o ambiente marítimo requer cuidados de **corrosão** e **umidade** (ISO 9223).

**Leitura recomendada e fontes (URL + DOI):**
- **ERA5‑Land monthly means (1991–2020):** **DOI:** https://doi.org/10.24381/cds.68d2bb30 — descrição do conjunto de dados e variáveis.
- **INMET** Normais 1991–2020: https://portal.inmet.gov.br/ — dados nacionais de clima (Brasil).
- **ISO 9223** (classificação de agressividade atmosférica marinha): https://www.iso.org/standard/120505.html
- **ZENG & ZHANG, 2010** (efeitos de T em AWE): **DOI:** https://doi.org/10.1016/j.pecs.2009.11.002
- **SONE et al., 1996** (condutividade da Nafion vs T): **DOI:** https://doi.org/10.1149/1.1836625